# code

In [ ]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
import cv2
from tensorflow.keras.layers import Conv2D
from keras.utils.np_utils import to_categorical


emotion_data = pd.read_csv('fer2013.csv')
#print(emotion_data)


X_train = []
y_train = []
X_test = []
y_test = []
for index, row in emotion_data.iterrows():
    k = row['pixels'].split(" ")
    if row['Usage'] == 'Training':
        X_train.append(np.array(k))
        y_train.append(row['emotion'])
    elif row['Usage'] == 'PublicTest':
        X_test.append(np.array(k))
        y_test.append(row['emotion'])
       
       
       
X_train = np.array(X_train,'float32')
y_train = np.array(y_train,'float32')
X_test = np.array(X_test,'float32')
y_test = np.array(y_test,'float32')

X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)
X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

y_train= to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)        

model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(48,48,1)))
model.add(Convolution2D(64,kernel_size = (3,3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64,kernel_size = (3,3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128,kernel_size = (3,3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128,kernel_size = (3,3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256,kernel_size = (3,3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256,kernel_size = (3,3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256,kernel_size = (3,3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512,kernel_size = (3,3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512,kernel_size = (3,3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512,kernel_size = (3,3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))


model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512,kernel_size = (3,3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512,kernel_size = (3,3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512,kernel_size = (3,3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))


model.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy'])

model.fit(X_train,y_train,batch_size=32,epochs=50,verbose=1,validation_data=(X_test, y_test))

loss_and_metrics = model.evaluate(X_test,y_test)
print(loss_and_metrics)


model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")
print("Saved model to disk")


import os
import cv2
import numpy as np
from keras.models import model_from_json
from keras.preprocessing import image
from tkinter import *

model = model_from_json(open("model.json", "r").read())
model.load_weights('model.h5')

facecasc = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
cap =cv2.VideoCapture(0)


while True:
    #_,frame=cv2.video_capture.read()

    ret, frame = cap.read()
    if not ret:
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)    

    faces_detected = facecasc.detectMultiScale(gray)
    for (x,y,w,h) in faces_detected:
        cv2.rectangle(frame,(x,y), (x+w,y+h), (0,255,0),thickness=2)
        roi_gray=gray[y:y+w,x:x+h]
        image_pixels = img_to_array(roi_gray)
        image_pixels = np.expand_dims(image_pixels,axis=0)
        image_pixels /= 255
        
        predictions = model.predict(image_pixels)        
        max_index = np.argmax(predictions[0])

        emotion_detection = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
        emotion_prediction = emotion_detection[max_index]

        cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
 
   
        #resized_image = cv2.resize(frame, (1000, 700))
        cv2.imshow('Emotion',resized_image)
        key = cv2.waitKey(1)
        if key == 27:
            break


cap.release()
cv2.destroyAllWindows